## 체인 여러개 연결

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


llm = ChatOllama(model="steamdj/llama3.1-cpu-only:latest")

# 첫번째 체인
prompt1 = ChatPromptTemplate.from_template("[{korean_input}] translate the question into English. Don't say anything else, just translate it.")
chain1 = (
    prompt1
    | llm
    | StrOutputParser()
)
message1 = chain1.invoke({"korean_input": "주식이 뭐야?"})
print(f'message1: {message1}') # What is a stock?

# 두번째 체인
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 똑똑이. answer the question in Korean"),
    ("user", "{input}")
])
chain2 = (
    {"input": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)
message2 = chain2.invoke({"korean_input": "주식이 뭐야?"})
print(f'message2: {message2}') # 주식은 한 회사의 소유권을 나타내는 증권입니다. 즉, 특정 기업에 투자하여 (중략)

**출력파서(Output Parser)**

LangChain의 출력파서는 언어 모델(LLM)의 출력을 더 유용하고 구조화된 형태로 변환하는 중요한 컴포넌트입니다.

**출력파서의 역할**

- LLM의 출력을 받아 더 적합한 형식으로 변환
- 구조화된 데이터 생성에 매우 유용
- LangChain 프레임워크에서 다양한 종류의 출력 데이터를 파싱하고 처리

**채팅 메시지를 문자열로 변환하는 출력 구문 분석기**

StrOutputParser, PydanticOuputParser, JsonOutputParser, StructuredOuputParser 등이 있음

- OutputParser 사용
    - 채팅 메시지를 문자열로 변환하는 출력 구문 분석기
        - StrOutputParser, PydanticOuputParser, JsonOutputParser, StructuredOuputParser 등
      

## 체인 병렬 실행

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOllama(model="steamdj/llama3.1-cpu-only:latest")

joke_chain = (
    ChatPromptTemplate.from_template("{topic}에 관련해서 짧은 농담 말해줘")
    | llm)
poem_chain = (
    ChatPromptTemplate.from_template("{topic}에 관련해서 시 2줄 써줘")
    | llm)

# map_chain = {"joke": joke_chain, "poem": poem_chain} # 체인에서 이처럼 사용할 때, 자동으로 RunnableParallel 사용됨
# map_chain = RunnableParallel({"joke": joke_chain, "poem": poem_chain})

from langchain_core.runnables import RunnableParallel
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

output = map_chain.invoke({"topic": "애플"})
print(output)

### OutputParser 사용 예

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()
chain.invoke({"input": "What is stock?"})

#### StrOutputParser

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="steamdj/llama3.1-cpu-only:latest")
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 똑똑이. Introduce yourself first, and answer the questions. answer me in Korean no matter what. "),
    ("user", "{input}")
])

from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()
output = chain.invoke({"input": "What is stock?"})
print(output)

#### CSV parser

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOllama(model="steamdj/llama3.1-cpu-only:latest")

chain = prompt | llm | output_parser

chain.invoke({"subject": "popular Korean cusine"})

#### JSON parser

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# 자료구조 정의 (pydantic)
class CusineRecipe(BaseModel):
    name: str = Field(description="name of a cusine")
    recipe: str = Field(description="recipe to cook the cusine")

# 출력 파서 정의
output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

#### Pydantic parser

In [ ]:
pip install langchain-teddynote

In [1]:
#Without Pydantic parser
from langchain_teddynote.messages import stream_response
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

llm = ChatOllama(model="llama3.1:latest")

email_conversation = """
From: 김상무 (sangmu.kim@greenmobility.co.kr)
To: 이대리 (daeri.lee@ecotechsolutions.com)
Subject: "E-STORE" 에너지 저장 장치 유통 협력 및 미팅 일정 제안

안녕하세요, 이대리 대리님,

저는 에코테크의김상무 상무입니다. 최근 보도자료를 통해 귀사의 신규 친환경 에너지 저장 장치 "E-STORE"에 대해 알게 되었>습니다. 에코테크는 친환경 에너지 솔루션과 관련된 기술 개발 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

E-STORE 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 저장 용량, 그리고 효율성 측면에 대한 정보가 필요합
니다. 이를 통해 저희가 제안할 솔루션 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나
 이야기를 나눌 수 있을까요?

감사합니다.

김상무
상무이사
에코테크

"""

from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)


chain = prompt | llm


answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)

ModuleNotFoundError: No module named 'langchain_teddynote'

In [ ]:
# With Pydantic parser
import re
import json
from langchain_teddynote.messages import stream_response
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

llm = ChatOllama(model="llama3.1:latest")

email_conversation = """
From: 김상무 (sangmu.kim@greenmobility.co.kr)
To: 이대리 (daeri.lee@ecotechsolutions.com)
Subject: "E-STORE" 에너지 저장 장치 유통 협력 및 미팅 일정 제안

안녕하세요, 이대리 대리님,

저는 에코테크의김상무 상무입니다. 최근 보도자료를 통해 귀사의 신규 친환경 에너지 저장 장치 "E-STORE"에 대해 알게 되었>습니다. 에코테크는 친환경 에너지 솔루션과 관련된 기술 개발 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

E-STORE 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 저장 용량, 그리고 효율성 측면에 대한 정보가 필요합
니다. 이를 통해 저희가 제안할 솔루션 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나
 이야기를 나눌 수 있을까요?

감사합니다.

김상무
상무이사
에코테크

"""

from itertools import chain
from langchain_core.prompts import PromptTemplate

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트", default="N/A")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간", default="N/A")

def preprocess_json(json_str):
    # 불필요한 공백 제거
    json_str = json_str.strip()

    # 필드 값의 시작과 끝에 있는 따옴표를 제거하고 내부의 따옴표를 이스케이프 처리
    def process_value(match):
        key = match.group(1)
        value = match.group(2).strip()
        if value.startswith('"') and value.endswith('"'):
            value = value[1:-1]
        # 값에 있는 모든 따옴표를 이스케이프 처리
        value = value.replace('"', '\\"')
        return f'"{key}": "{value}"'

    # 모든 필드의 값을 처리
    json_str = re.sub(r'"(\w+)":\s*(.+?)(?=,|\s*}|$)', process_value, json_str)

    # 마지막 필드 뒤에 쉼표가 있다면 제거
    json_str = re.sub(r',\s*}$', '}', json_str)

    return json_str

# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)
# instruction 을 출력합니다.
print(parser.get_format_instructions())

#You are a helpful assistant. Please answer the following questions in KOREAN.

prompt = PromptTemplate.from_template(
"""
You are a helpful assistant. Extract the main contents of the email and provide the result as a valid JSON object.
Your response should contain ONLY the JSON object, without any additional text or explanation.
Ensure that the JSON is properly formatted, with all values enclosed in double quotes and any internal quotes escaped.
Do not use line breaks within the JSON object.
The JSON MUST include exactly these keys: "person", "email", and "subject".
If information for any field is not available, use "N/A" as the value.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}

RESPONSE (ONLY JSON):
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

chain = prompt | llm


# chain 을 실행하고 결과를 출력합니다.
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)


# 유니코드 제어 문자 제거 함수
import unicodedata

def remove_control_characters(s):
    return "".join(ch for ch in s if unicodedata.category(ch)[0] != "C")

# LLM 출력 후 전처리 및 파싱
output = stream_response(response, return_output=True)
output = remove_control_characters(output)
print("Raw LLM output:")
print(output)

try:
    preprocessed_output = preprocess_json(output)
    print("Preprocessed output:")
    print(preprocessed_output)

    json_data = json.loads(preprocessed_output)

    structured_output = EmailSummary(**json_data)
    print("Structured output:")
    print(structured_output)
except json.JSONDecodeError as e:
    print(f"JSON Decode Error: {e}")
    print("Preprocessed output:")
    print(preprocessed_output)
except ValidationError as e:
    print(f"Pydantic Validation Error: {e}")
    print("JSON data:")
    print(json_data)
except Exception as e:
    print(f"Error: {type(e).__name__} - {e}")
    print("Raw output:")
    print(output)

### HuggingFace-Hub 설치

In [ ]:
pip install huggingface-hub

**모델 다운로드**

명령어 형식:  `HuggingFace Repo` \ .gguf 파일명\ local-dir 설정 \심볼릭 링크 설정

In [ ]:
huggingface-cli download \
  heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF \
  ggml-model-Q5_K_M.gguf \
  --local-dir [본인의_컴퓨터_다운로드폴더_경로] \
  --local-dir-use-symlinks False

Modelfile 예시

In [ ]:
FROM ggml-model-Q5_K_M.gguf

TEMPLATE """{{- if .System }}
<s>{{ .System }}</s>
{{- end }}
<s>Human:
{{ .Prompt }}</s>
<s>Assistant:
"""
# 시스템 메시지 설정
SYSTEM """A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."""

TEMPERATURE 0 # 온도 설정 (높을수록 창의적, 낮을수록 정확함)
PARAMETER stop <s>
PARAMETER stop </s>

###  모델파일을 기준으로  LLM(gguf) 설치하기

In [ ]:
# ollma create [만들이름] -f [모델파일경로]
ollama create Test1-10.8B -f EEVE-Korean-Instruct-10.8B-v1.0-GGUF/Modelfile

In [ ]:
$ pip install langchain langchain-community langchain-core
$ pip install langchain-openai sse_starlette pydantic==1.10.13
$ pip install sse_starlette
$ pip install pydantic==1.10.13

라이브러리 추가 설치